In [11]:
import os
import json
import random
from pathlib import Path
from typing import List, Dict

# filtering patterns
starts = [
    'This website is using a security service to protect',
    'To continue, please click the box',
    'ARTICLE_FETCH_FAILED:',
    'Sorry the page'
]

random.seed(42)  # for reproducibility


In [12]:
def is_valid_story(text: str, invalid_starts: List[str]) -> bool:
    """Return False if story starts with any invalid prefix."""
    return not any(text.strip().startswith(s) for s in invalid_starts)


def preprocess_entry(entry: Dict, invalid_starts: List[str]) -> List[Dict]:
    """Convert one article entry into multiple instruction-input-output samples."""
    headline = entry.get("headline", "").strip()
    results = []
    
    for stance in ["left", "center", "right"]:
        stories = entry.get("story", {}).get(stance, [])
        for s in stories:
            if not is_valid_story(s, invalid_starts):
                continue
            s = s.strip()
            if not s:
                continue
                
            formatted = {
                "instruction": f"Write a political news story from a {stance} perspective based on the headline.",
                "input": f"Headline: {headline}",
                "output": s
            }
            results.append(formatted)
    return results

In [13]:
def process_dataset(json_path: str, invalid_starts: List[str]) -> List[Dict]:
    """Load raw JSON, filter, flatten, and return processed examples."""
    with open(json_path, "r", encoding="utf-8") as f:
        raw_data = json.load(f)

    processed = []
    for i, entry in enumerate(raw_data):
        examples = preprocess_entry(entry, invalid_starts)
        processed.extend(examples)
        
        if i % 200 == 0:
            print(f"Processed {i} entries, total samples: {len(processed)}")

    print(f"\n✅ Finished processing {len(raw_data)} raw entries into {len(processed)} samples.")
    return processed

In [14]:
def split_and_save(data: List[Dict], output_dir: str, train_ratio: float = 0.8):
    """Split into train/validation sets and save as JSONL."""
    random.shuffle(data)
    split_idx = int(len(data) * train_ratio)
    train_data = data[:split_idx]
    val_data = data[split_idx:]
    
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)

    def save_jsonl(filename: str, samples: List[Dict]):
        with open(output_path / filename, "w", encoding="utf-8") as f:
            for sample in samples:
                f.write(json.dumps(sample, ensure_ascii=False) + "\n")

    save_jsonl("train.jsonl", train_data)
    save_jsonl("val.jsonl", val_data)

    print(f"✅ Saved {len(train_data)} training samples and {len(val_data)} validation samples to {output_path}")

In [15]:
# start from current working directory
cwd = os.getcwd()

# build full paths
raw_json_path = os.path.join(cwd,"..", "data", "data.json")   # input file
output_dir = os.path.join(cwd,"..", "data", "processed")      # output folder

print(f"📂 Current working directory: {cwd}")
print(f"📄 Input file: {raw_json_path}")
print(f"📁 Output directory: {output_dir}")

# run pipeline
processed_data = process_dataset(raw_json_path, starts)
split_and_save(processed_data, output_dir, train_ratio=0.8)

📂 Current working directory: /Users/xuyanshuo/projects/AI-Political-Perspectives/training
📄 Input file: /Users/xuyanshuo/projects/AI-Political-Perspectives/training/../data/data.json
📁 Output directory: /Users/xuyanshuo/projects/AI-Political-Perspectives/training/../data/processed
Processed 0 entries, total samples: 1
Processed 200 entries, total samples: 452
Processed 400 entries, total samples: 869
Processed 600 entries, total samples: 1297
Processed 800 entries, total samples: 1668
Processed 1000 entries, total samples: 2137
Processed 1200 entries, total samples: 2565
Processed 1400 entries, total samples: 3002
Processed 1600 entries, total samples: 3437

✅ Finished processing 1664 raw entries into 3565 samples.
✅ Saved 2852 training samples and 713 validation samples to /Users/xuyanshuo/projects/AI-Political-Perspectives/training/../data/processed
